In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
from scikeras.wrappers import KerasClassifier


In [ ]:
from sklearn.model_selection import GridSearchCV

In [11]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle as pkl

In [20]:
data=pd.read_csv("Churn_Modelling.csv")

data.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

ohe_geo=OneHotEncoder()
geo_encoder = ohe_geo.fit_transform(data[['Geography']]).toarray()

geo_encoder_df=pd.DataFrame(geo_encoder, columns=ohe_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df], axis=1)

##divide the dataset into dependent and independent features
x=data.drop('Exited',axis=1)  ###input is all columns except 'Exited'
y=data['Exited']   ##output is 'Exited' column

##split the dataset into training and testing sets
X_train, X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

##scale the features
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [21]:
##save the encoder and sscalar
with open('label_encoder_gender.pkl','wb') as file:
    pkl.dump(label_encoder_gender,file)

with open('ohe_geo.pkl','wb') as file:
    pkl.dump(ohe_geo,file)

with open('scaler.pkl','wb') as file:
    pkl.dump(scaler,file)

In [ ]:


# Create the model function with **kwargs to accept parameters passed by scikeras
def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model 

# Initialize KerasClassifier without specifying neurons/layers here
model = KerasClassifier(model=create_model, verbose=0)

# Define the hyperparameter grid (these are parameters of the model function)
param_grid = {
    'model__neurons': [16, 32, 64, 128],   # Notice the 'model__' prefix
    'model__layers': [1, 2],
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)

# Fit the grid search
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


Fitting 3 folds for each of 32 candidates, totalling 96 fits


KeyboardInterrupt: 